In [31]:

def fixDataLength(path):
    '''
    gets rid of the weird length stuff
    path (string) : filepath to the .log file for a participant
    '''     
    x = open(path)
    s = x.read().replace(',,,', ',NaN,NaN,')
    x.close()

    # Now open the file in write mode and write the string with replaced

    x=open(path,'w')
    x.write(s)
    x.close()
    

In [42]:
def getAccAndRT(df):
    '''
    returns [array]: [reaction time, accuracy] for dataframe df
    df needs to have ['respTime'] and ['accuracy'] columns, and a name
    '''
    accCount = df['accuracy'].astype(int).sum()

    accPerc = accCount / len(df)
    RTdf = df.loc[df['accuracy'] == 1] #remove mistake trials for RT calculation
    RT = RTdf['respTime'].astype(float).mean()
    
    print(df.name + ' AccPerc is ' + str(accPerc))
    print(df.name + ' avg resp time: ' + str(RT))
    
    return [accPerc, RT]



In [68]:
import pandas as pd
import glob
import os
import copy

#global script variables
accCutoff = .65
path = r'/Users/jinjiang-macair/Library/CloudStorage/OneDrive-DukeUniversity/Duke/Fall2022Rotation/Data/contextFlanker2.0/'

#load files from path
allFiles = glob.glob(path + "*.txt")
assignmentID = allFiles
subjectList = []
assignmentList = []
groupDataList = []
aboveAccCutoff = []
accList = []

for file in allFiles:
    # open participant info file
    partInfoFile = open(file,"r").readlines()
    partInfo = [x.strip() for x in partInfoFile]
    
    #pull worker ID and assignment ID from file
    wID = partInfo[2]
    aID = partInfo[1]
    compID = wID.split(':')  #getting worker ID
    compID2 = aID.split(':') #getting assignment ID
    subject = compID[1] #subID
    
    #open log file with the same name as the given particant info file
    logfile = compID2[1]+'.log'
    
    #fixDataLength(path+logfile) #dunno if i should return the log file instead of doing a void fxn here
    
    partDataFile = pd.read_csv(path + logfile, lineterminator=';', sep=',', header=None, error_bad_lines = False);
    partData = pd.concat([partDataFile], ignore_index=True, axis = 1) 
    testData = copy.deepcopy(partData)
    
    #formatting result dataframe
    
    # these are longer than they seem.
    partData['subject']=subject #adding a column with subID
    partData = partData.rename(columns={0: 'expStage', 1: 'sectionType', 2: 'taskName', 3: 'blockNumber', 4: 'firstBlockName', 5: 'secondBlockName', 6: 'thirdBlockName', 
                                        7:'currentBlockName', 8: 'trialCount', 9: 'blockTrialCount', 10: 'flankerSize',
                                        11: 'trialImg', 12: 'imgSource', 13:'imgCongruency', 14: 'imgDisplayLocation', 15:'repeat', 
                                        16: 'stimOnset', 17: 'respOnset', 18: 'respTime', 19: 'accuracy', 20: 'sectionStart', 
                                        21: 'sectionEnd', 22: 'sectionEnd-sectionStart', 23:'subject'}) 
       
    #add info to individual lists
    subjectList.append(subject)
    assignmentList.append(compID2[1])
    
    #add data to whole sample dataframe list
    groupDataList.append(partData)

    #save out participant data to CSV if it doesn't exist already
    if os.path.exists(path+'Subj_'+str(subject)+'.csv'):
        print(subject+' exists.') #don't make csv again if it already exits (file creation times are helpful for keeping track of data)
    else:
        print('Creating csv file for '+subject)
        partData.to_csv(path+'Subj_'+str(subject)+'.csv', index=False)
    
#combine groupData into one file and save file
groupData = pd.concat(groupDataList, ignore_index = True)

# save out groupData into csv
print('Creating group level csv file.')
groupData.to_csv(path+'combinedData'+'.csv', index=False)

am5tnxthwn6pz exists.
aw5o1rk3w60fc exists.
Creating csv file for a1msg3v72sbkvu
Creating csv file for a3lxaru55pr281
a1j980qnbdg17z exists.
Creating csv file for a25pfsordo3swq
a12b7sh0t31k8u exists.
Creating csv file for a31nsutwp9a51h
a3uaxlnvxordwj exists.
Creating csv file for a2w6qfkxrq5fcp
Creating csv file for a3nxt3ovgl7qnr
a32j7anzoukv9n exists.
awa928gj41l3a exists.
Creating csv file for a1qcfzj4pfm9wi
a3b2yrsmsc2j3t exists.
aqaxdl1inqc9q exists.
a1h198mrim37t1 exists.
Creating csv file for a3ckwbnfp1zhbl
a1m64r8hpynnza exists.
a22cwnsgo4uvgo exists.
aiyomf8pjjleq exists.
Creating csv file for a1ime6dsk137x4
awtcgy2cl1t68 exists.
aiekcwyzts41v exists.
a3q7072s2ngysl exists.
ajq71yigy01hz exists.
a2efmn1afprcl exists.
Creating csv file for a1t6dmrdvola7x
Creating group level csv file.


In [69]:
allData = pd.read_csv('/Users/jinjiang-macair/Library/CloudStorage/OneDrive-DukeUniversity/Duke/Fall2022Rotation/Data/contextFlanker2.0/combinedData.csv')

allData = allData.loc[allData['sectionType'] == 'main1']

In [70]:
import pandas as pd

accThreshold = 75

highAccDF = pd.DataFrame()

#subjectList.remove('a3ljsvb3hlepnw')

for subject in subjectList:
    subjectDF = allData.loc[allData['subject'] == subject]
    
    '''
    Check if the accuracy of each subject is above our threshold.
    If so, add it to the highAccDF.
    '''
    if (100 * subjectDF['accuracy'].astype(int).sum() / len(subjectDF)) > accThreshold:
        highAccDF = pd.concat([highAccDF, subjectDF])
    




In [71]:
conDF = highAccDF.loc[(highAccDF['imgCongruency'] == 'c')]
conDF.name = 'conDF'

incDF = highAccDF.loc[(highAccDF['imgCongruency'] == 'i')]
incDF.name = 'incDF'

getAccAndRT(conDF)
getAccAndRT(incDF)





conDF AccPerc is 0.909280303030303
conDF avg resp time: 1013.9954176213289
incDF AccPerc is 0.8880681818181818
incDF avg resp time: 1034.744934954148


[0.8880681818181818, 1034.744934954148]

In [79]:
# need to grab the respTime and Acc for each trial AFTER a congruent trial


# make an empty dataframe for trials AFTER congruent ones
# make another empty dataframe for trials after incongruent ones

# for each row in the test block (trialCount is > 200)
    # check if it's congruent or incongruent
        # if congruent
            # grab next row and add to the afterCongruent dataframe
        # if incongruent
            # grab next row and add to the afterIncongruent dataframe
            
# testBlockDF = highAccDF.loc[highAccDF['currentBlockName'].isnull()] #this is only for the first batch of participants cuz it was messed up

testBlockDF = highAccDF.loc[highAccDF['currentBlockName'] == 'test']

conTestDF = testBlockDF.loc[testBlockDF['imgCongruency'] == 'c']
incTestDF = testBlockDF.loc[testBlockDF['imgCongruency'] == 'i']

conTestDF.name = 'conTestDF'
incTestDF.name = 'incTestDF'

afterConTestDF = testBlockDF.loc[testBlockDF['imgCongruency'].shift(1) == 'c'] # can you just do this
afterIncTestDF = testBlockDF.loc[testBlockDF['imgCongruency'].shift(1) == 'i']

afterConTestDF.name = 'afterConTestDF'
afterIncTestDF.name = 'afterIncTestDF'


In [80]:
getAccAndRT(afterConTestDF)

getAccAndRT(afterIncTestDF)

getAccAndRT(conTestDF)

getAccAndRT(incTestDF)



afterConTestDF AccPerc is 0.9065647794373046
afterConTestDF avg resp time: 1023.6298850574713
afterIncTestDF AccPerc is 0.9090277777777778
afterIncTestDF avg resp time: 1026.1646294881589
conTestDF AccPerc is 0.9184027777777778
conTestDF avg resp time: 1013.7251417769376
incTestDF AccPerc is 0.8972222222222223
incTestDF avg resp time: 1036.3014705882354


[0.8972222222222223, 1036.3014705882354]

In [77]:
highAccDF.loc[highAccDF['blockNumber'] == 1]

,expStage,sectionType,taskName,blockNumber,firstBlockName,secondBlockName,thirdBlockName,currentBlockName,trialCount,blockTrialCount,...,imgDisplayLocation,repeat,stimOnset,respOnset,respTime,accuracy,sectionStart,sectionEnd,sectionEnd-sectionStart,subject
65,NaN,main1,contextFlankerMainTask,1.0,learnBtop,learnAbot,test,learnBtop,1.0,1.0,...,top,True,560871.0,562193.0,1322.0,1.0,NaN,NaN,NaN,am5tnxthwn6pz
66,NaN,main1,contextFlankerMainTask,1.0,learnBtop,learnAbot,test,learnBtop,2.0,2.0,...,top,True,564095.0,565218.0,1123.0,1.0,NaN,NaN,NaN,am5tnxthwn6pz
67,NaN,main1,contextFlankerMainTask,1.0,learnBtop,learnAbot,test,learnBtop,3.0,3.0,...,top,True,567024.0,567881.0,857.0,1.0,NaN,NaN,NaN,am5tnxthwn6pz
68,NaN,main1,contextFlankerMainTask,1.0,learnBtop,learnAbot,test,learnBtop,4.0,4.0,...,top,True,569789.0,570641.0,852.0,1.0,NaN,NaN,NaN,am5tnxthwn6pz
69,NaN,main1,contextFlankerMainTask,1.0,learnBtop,learnAbot,test,learnBtop,5.0,5.0,...,top,True,572457.0,573274.0,817.0,1.0,NaN,NaN,NaN,am5tnxthwn6pz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,NaN,main1,contextFlankerMainTask,1.0,learnBbot,learnAtop,test,learnBbot,96.0,96.0,...,bottom,True,674673.0,676062.0,1389.0,1.0,NaN,NaN,NaN,a1t6dmrdvola7x
12663,NaN,main1,contextFlankerMainTask,1.0,learnBbot,learnAtop,test,learnBbot,97.0,97.0,...,bottom,True,678151.0,679738.0,1587.0,1.0,NaN,NaN,NaN,a1t6dmrdvola7x
12664,NaN,main1,contextFlankerMainTask,1.0,learnBbot,learnAtop,test,learnBbot,98.0,98.0,...,bottom,True,681694.0,683163.0,1469.0,1.0,NaN,NaN,NaN,a1t6dmrdvola7x
12665,NaN,main1,contextFlankerMainTask,1.0,learnBbot,learnAtop,test,learnBbot,99.0,99.0,...,bottom,True,685074.0,686200.0,1126.0,1.0,NaN,NaN,NaN,a1t6dmrdvola7x


In [78]:
firstLearnBlockHighConDF = highAccDF.loc[(highAccDF['blockNumber'] == 1) & (highAccDF['firstBlockName'].isin(['learnBtop', 'learnBbot']))]
firstLearnBlockHighConDF.name = 'firstLearnBlockHighConDF'

secondLearnBlockHighConDF = highAccDF.loc[(highAccDF['blockNumber'] == 2) & (highAccDF['secondBlockName'].isin(['learnBtop', 'learnBbot']))]
secondLearnBlockHighConDF.name = 'secondLearnBlockHighConDF'

firstLearnBlockLowConDF = highAccDF.loc[(highAccDF['blockNumber'] == 1) & (highAccDF['firstBlockName'].isin(['learnAtop', 'learnAbot']))]
firstLearnBlockLowConDF.name = 'firstLearnBlockLowConDF'

secondLearnBlockLowConDF = highAccDF.loc[(highAccDF['blockNumber'] == 2) & (highAccDF['secondBlockName'].isin(['learnAtop', 'learnAbot']))]
secondLearnBlockLowConDF.name = 'secondLearnBlockLowConDF'

learnBlocksHighConDF = pd.concat([firstLearnBlockHighConDF, secondLearnBlockHighConDF])
learnBlocksHighConDF.name = 'learnBlocksHighConDF'

learnBlocksLowConDF = pd.concat([firstLearnBlockLowConDF, secondLearnBlockLowConDF])
learnBlocksLowConDF.name = 'learnBlocksLowConDF'

learnBlocksLowConCongruentTrialsDF = learnBlocksLowConDF.loc[learnBlocksLowConDF['imgCongruency'] == 'c']
learnBlocksLowConCongruentTrialsDF.name = 'learnBlocksLowConCongruentTrialsDF'

learnBlocksLowConIncongruentTrialsDF = learnBlocksLowConDF.loc[learnBlocksLowConDF['imgCongruency'] == 'i']
learnBlocksLowConIncongruentTrialsDF.name = 'learnBlocksLowConIncongruentTrialsDF'

learnBlocksHighConCongruentTrialsDF = learnBlocksHighConDF.loc[learnBlocksHighConDF['imgCongruency'] == 'c']
learnBlocksHighConCongruentTrialsDF.name = 'learnBlocksHighConCongruentTrialsDF'

learnBlocksHighConIncongruentTrialsDF = learnBlocksHighConDF.loc[learnBlocksHighConDF['imgCongruency'] == 'i']
learnBlocksHighConIncongruentTrialsDF.name = 'learnBlocksHighConIncongruentTrialsDF'

getAccAndRT(learnBlocksHighConDF)
getAccAndRT(learnBlocksLowConDF)

getAccAndRT(learnBlocksHighConCongruentTrialsDF)
getAccAndRT(learnBlocksHighConIncongruentTrialsDF)

getAccAndRT(learnBlocksLowConCongruentTrialsDF)
getAccAndRT(learnBlocksLowConIncongruentTrialsDF)


learnBlocksHighConDF AccPerc is 0.8929166666666667
learnBlocksHighConDF avg resp time: 1021.0522631824545
learnBlocksLowConDF AccPerc is 0.8825
learnBlocksLowConDF avg resp time: 1025.9159584513693
learnBlocksHighConCongruentTrialsDF AccPerc is 0.9005208333333333
learnBlocksHighConCongruentTrialsDF avg resp time: 1014.733950260266
learnBlocksHighConIncongruentTrialsDF AccPerc is 0.8625
learnBlocksHighConIncongruentTrialsDF avg resp time: 1047.4396135265702
learnBlocksLowConCongruentTrialsDF AccPerc is 0.8895833333333333
learnBlocksLowConCongruentTrialsDF avg resp time: 1012.679156908665
learnBlocksLowConIncongruentTrialsDF AccPerc is 0.8807291666666667
learnBlocksLowConIncongruentTrialsDF avg resp time: 1029.258426966292


[0.8807291666666667, 1029.258426966292]